In [40]:
from openai import AzureOpenAI
import time
from IPython.display import clear_output
import json

StatementMeta(, a654f3ea-412f-4d3d-8171-c5ef709afc92, 44, Finished, Available)

In [41]:
azure_openai_endpoint = mssparkutils.credentials.getSecret('https://algdev-scannerapi.vault.azure.net/','azure-openai-endpoint')
azure_openai_api_key = mssparkutils.credentials.getSecret('https://algdev-scannerapi.vault.azure.net/','azure-openai-api-key')


StatementMeta(, a654f3ea-412f-4d3d-8171-c5ef709afc92, 45, Finished, Available)

In [42]:
#https://learn.microsoft.com/en-us/azure/ai-services/openai/how-to/assistant

# Initialize the Azure OpenAI client
client = AzureOpenAI(
        azure_endpoint = azure_openai_endpoint, 
        api_key=azure_openai_api_key,  
        api_version="2024-02-15-preview"
        )

StatementMeta(, a654f3ea-412f-4d3d-8171-c5ef709afc92, 46, Finished, Available)

In [32]:
assistant = client.beta.assistants.create(
  instructions="You are an AI coding assistant that can help translate powershell code into python",
  model="gpt-4-1106-preview", # replace with model deployment name. 
  tools=[{"type": "code_interpreter"}]
)

StatementMeta(, a654f3ea-412f-4d3d-8171-c5ef709afc92, 36, Finished, Available)

In [33]:
# Create an assistant
assistant = client.beta.assistants.create(
    name="Python Translator",
    instructions=f"You are a coding specalist who takes powershell code as an input and transforms the code into python. \
    Execute each of the steps listed below in your ACTIONS section." 
    f"ACTIONS:"
    f"1. You have access to a sandboxed environment for writing and testing code."
    f"2. Read the entire powershell code"
    f"3. Translate the powershell code into python"
    f"4. Use python best practices when translating the code"
    f"5. Run the code to confirm that it runs."
    f"6. If the code is successful output the code and provide a summary paragraph explaining the preparation of the code translation."
    f"7. If the code is unsuccessful display the error message and try to revise the code and rerun going through the steps from above again."
    f"8. Output the new python code.",
    tools=[{"type": "code_interpreter"}],
    model="gpt-4-1106-preview" #You must replace this value with the deployment name for your model.
)


StatementMeta(, a654f3ea-412f-4d3d-8171-c5ef709afc92, 37, Finished, Available)

In [34]:
# Create a thread
thread = client.beta.threads.create()

StatementMeta(, a654f3ea-412f-4d3d-8171-c5ef709afc92, 38, Finished, Available)

In [35]:
# Add a user question to the thread
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content=
    
    """Translate the following powershell code into python: 

    #Requires -Modules MicrosoftPowerBIMgmt.Profile

    param(               
        [psobject]$config
        ,
        [string]$stateFilePath     
    )

    try { 
        Write-Host "Starting Power BI Activity Fetch"

        $stopwatch = [System.Diagnostics.Stopwatch]::new()
        $stopwatch.Start()   

        if ($config.ActivityFileBatchSize)
        {
            $outputBatchCount = $config.ActivityFileBatchSize
        }
        else {
            $outputBatchCount = 5000   
        }    

        $rootOutputPath = "$($config.OutputPath)\activity"
        New-Item -ItemType Directory -Path $rootOutputPath -ErrorAction SilentlyContinue | Out-Null

        $outputPath = "$rootOutputPath\{0:yyyy}\{0:MM}"    
        
        if (!$stateFilePath) {
            $stateFilePath = "$($config.OutputPath)\state.json"
        }

        if (Test-Path $stateFilePath) {
            $state = Get-Content $stateFilePath | ConvertFrom-Json
        }
        else {
            $state = New-Object psobject 
        }
        
        $maxHistoryDate = [datetime]::UtcNow.Date.AddDays(-30)

        if ($state.Activity.LastRun) {
            if (!($state.Activity.LastRun -is [datetime])) {
                $state.Activity.LastRun = [datetime]::Parse($state.Activity.LastRun).ToUniversalTime()
            }
            $pivotDate = $state.Activity.LastRun
        }
        else {
            $state | Add-Member -NotePropertyName "Activity" -NotePropertyValue @{"LastRun" = $null } -Force
            $pivotDate = $maxHistoryDate
        }

        if ($pivotDate -lt $maxHistoryDate)
        {
            Write-Host "Last run was more than 30 days ago"
            $pivotDate = $maxHistoryDate
        }

        Write-Host "Since: $($pivotDate.ToString("s"))"
        Write-Host "OutputBatchCount: $outputBatchCount"

        Write-Host "Getting OAuth Token"

        if ($config.ServicePrincipal.AppId)
        {
            $credential = New-Object -TypeName System.Management.Automation.PSCredential -ArgumentList $config.ServicePrincipal.AppId, ($config.ServicePrincipal.AppSecret | ConvertTo-SecureString -AsPlainText -Force)

            $pbiAccount = Connect-PowerBIServiceAccount -ServicePrincipal -Tenant $config.ServicePrincipal.TenantId -Credential $credential -Environment $config.ServicePrincipal.Environment
        }
        else {
            $pbiAccount = Connect-PowerBIServiceAccount
        }

        Write-Host "Login with: $($pbiAccount.UserName)"
        
        # Gets audit data for each day

        while ($pivotDate -le [datetime]::UtcNow) {           
            Write-Host "Getting audit data for: '$($pivotDate.ToString("yyyyMMdd"))'"        
                
            $activityAPIUrl = "admin/activityevents?startDateTime='$($pivotDate.ToString("s"))'&endDateTime='$($pivotDate.AddHours(24).AddSeconds(-1).ToString("s"))'"

            $audits = @()                  
            $pageIndex = 1
            $flagNoActivity = $true

            do
            {          
                if (!$result.continuationUri)
                {
                    $result = Invoke-PowerBIRestMethod -Url $activityAPIUrl -method Get | ConvertFrom-Json
                }
                else {
                    $result = Invoke-PowerBIRestMethod -Url $result.continuationUri -method Get | ConvertFrom-Json
                }            
                                    
                if ($result.activityEventEntities)
                {
                    $audits += @($result.activityEventEntities)               
                }

                if ($audits.Count -ne 0 -and ($audits.Count -ge $outputBatchCount -or $result.continuationToken -eq $null))
                {
                    # To avoid duplicate data on existing files, first dont append pageindex to overwrite existing full file

                    if ($pageIndex -eq 1)
                    {
                        $outputFilePath = ("$outputPath\{0:yyyyMMdd}.json" -f $pivotDate)                        
                    }
                    else {
                        $outputFilePath = ("$outputPath\{0:yyyyMMdd}_$pageIndex.json" -f $pivotDate)
                    }                    

                    Write-Host "Writing '$($audits.Count)' audits"

                    New-Item -Path (Split-Path $outputFilePath -Parent) -ItemType Directory -Force -ErrorAction SilentlyContinue | Out-Null
        
                    ConvertTo-Json @($audits) -Compress -Depth 10 | Out-File $outputFilePath -force

                    if ($config.StorageAccountConnStr -and (Test-Path $outputFilePath)) {
                        Write-Host "Writing to Blob Storage"
                        
                        $storageRootPath = "$($config.StorageAccountContainerRootPath)/activity"
            
                        Add-FileToBlobStorage -storageAccountConnStr $config.StorageAccountConnStr -storageContainerName $config.StorageAccountContainerName -storageRootPath $storageRootPath -filePath $outputFilePath -rootFolderPath $rootOutputPath         

                        Write-Host "Deleting local file '$outputFilePath'"

                        Remove-Item $outputFilePath -Force
                    }
                    
                    $flagNoActivity = $false

                    $pageIndex++

                    $audits = @()
                }
            }
            while($result.continuationToken -ne $null)

            if ($flagNoActivity)
            {
                Write-Warning "No audit logs for date: '$($pivotDate.ToString("yyyyMMdd"))'"
            }    

            $state.Activity.LastRun = $pivotDate.Date.ToString("o")

            $pivotDate = $pivotDate.AddDays(1)

            # Save state 

            Write-Host "Saving state"
            
            New-Item -Path (Split-Path $stateFilePath -Parent) -ItemType Directory -Force -ErrorAction SilentlyContinue | Out-Null
            
            ConvertTo-Json $state | Out-File $stateFilePath -force -Encoding utf8        
        }

    }
    finally {
        $stopwatch.Stop()

        Write-Host "Ellapsed: $($stopwatch.Elapsed.TotalSeconds)s"
    }

"""
)

StatementMeta(, a654f3ea-412f-4d3d-8171-c5ef709afc92, 39, Finished, Available)

In [36]:
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
  #instructions="New instructions" #You can optionally provide new instructions but these will override the default instructions
)

StatementMeta(, a654f3ea-412f-4d3d-8171-c5ef709afc92, 40, Finished, Available)

In [37]:
start_time = time.time()

status = run.status

while status not in ["completed", "cancelled", "expired", "failed"]:
    time.sleep(5)
    run = client.beta.threads.runs.retrieve(thread_id=thread.id,run_id=run.id)
    print("Elapsed time: {} minutes {} seconds".format(int((time.time() - start_time) // 60), int((time.time() - start_time) % 60)))
    status = run.status
    print(f'Status: {status}')
    clear_output(wait=True)

messages = client.beta.threads.messages.list(
  thread_id=thread.id
) 

print(f'Status: {status}')
print("Elapsed time: {} minutes {} seconds".format(int((time.time() - start_time) // 60), int((time.time() - start_time) % 60)))
print(messages.model_dump_json(indent=2))

Elapsed time: 4 minutes 0 seconds
Status: completed
Status: completed
Elapsed time: 4 minutes 1 seconds
{
  "data": [
    {
      "id": "msg_09IpmC16v21zqlFlfVzqIJIh",
      "assistant_id": "asst_MmlGX9fyEN0gYoLCEVdf8aaQ",
      "content": [
        {
          "text": {
            "annotations": [],
            "value": "The modified script section with the corrected output path ran successfully in the sandboxed environment. To summarize:\n\n- I translated the PowerShell script into Python, utilizing Python best practices.\n- I created a placeholder `config` dictionary and `state_file_path` to simulate the script's inputs.\n- External cmdlets were replaced with placeholder Python functions to indicate where you would include interactions with external systems or services (for instance, Azure blob storage or Power BI).\n- The directory creation operation initially failed due to a permissions issue since the provided path was not writable in our environment. I corrected this by adjusti

In [38]:
messages = client.beta.threads.messages.list(
  thread_id=thread.id
)

print(messages.model_dump_json(indent=2))

StatementMeta(, a654f3ea-412f-4d3d-8171-c5ef709afc92, 42, Finished, Available)

{
  "data": [
    {
      "id": "msg_09IpmC16v21zqlFlfVzqIJIh",
      "assistant_id": "asst_MmlGX9fyEN0gYoLCEVdf8aaQ",
      "content": [
        {
          "text": {
            "annotations": [],
            "value": "The modified script section with the corrected output path ran successfully in the sandboxed environment. To summarize:\n\n- I translated the PowerShell script into Python, utilizing Python best practices.\n- I created a placeholder `config` dictionary and `state_file_path` to simulate the script's inputs.\n- External cmdlets were replaced with placeholder Python functions to indicate where you would include interactions with external systems or services (for instance, Azure blob storage or Power BI).\n- The directory creation operation initially failed due to a permissions issue since the provided path was not writable in our environment. I corrected this by adjusting the path to point to the writable `/mnt/data` directory.\n\nFor real-world use, you would need to imp

In [39]:
data = json.loads(messages.model_dump_json(indent=2))
code = data['data'][0]['content'][0]['text']['value']
print(code)

StatementMeta(, a654f3ea-412f-4d3d-8171-c5ef709afc92, 43, Finished, Available)

The modified script section with the corrected output path ran successfully in the sandboxed environment. To summarize:

- I translated the PowerShell script into Python, utilizing Python best practices.
- I created a placeholder `config` dictionary and `state_file_path` to simulate the script's inputs.
- External cmdlets were replaced with placeholder Python functions to indicate where you would include interactions with external systems or services (for instance, Azure blob storage or Power BI).
- The directory creation operation initially failed due to a permissions issue since the provided path was not writable in our environment. I corrected this by adjusting the path to point to the writable `/mnt/data` directory.

For real-world use, you would need to implement the logic within placeholder functions using appropriate Python libraries. Here is the complete code snippet that successfully executed:

```python
import json
import os
from datetime import datetime, timedelta
import req